In [ ]:
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Accessing the Electricity Demand Dataset

This notebook assumes that you have downloaded the [electricity-demand](https://huggingface.co/datasets/EDS-lab/electricity-demand/tree/main/data) dataset from HuggingFace. To execute this notebook create a folder `data/electricity-demand/` and download the 3 files into it.

In [ ]:
from enfobench.datasets import ElectricityDemandDataset

ds = ElectricityDemandDataset("../data/electricity-demand")

In [ ]:
unique_ids = ds.metadata_subset.list_unique_ids()
unique_id = unique_ids[0]

In [ ]:
target, weather, metadata = ds.get_data_by_unique_id(unique_id)

In [ ]:
target.info()

In [ ]:
weather.info()

In [ ]:
from enfobench.datasets.utils import create_perfect_forecasts_from_covariates

In [ ]:
perfect_weather_forecasts = create_perfect_forecasts_from_covariates(
    weather[['temperature_2m', 'relative_humidity_2m', 'wind_speed_10m', 'wind_direction_10m', 'cloud_cover']],
    start=pd.Timestamp("2013-01-01T00:00:00"),
    horizon=pd.Timedelta("4 days"),
    step=pd.Timedelta("24 hour"),
)

In [ ]:
perfect_weather_forecasts.info()

# Create Dataset

In [ ]:
from enfobench import Dataset

dataset = Dataset(
    target=target,
    past_covariates=weather,
    future_covariates=perfect_weather_forecasts,
    metadata=metadata,
)

# Perform crossvalidation via server

In [ ]:
from enfobench.evaluation import ForecastClient

client = ForecastClient(port=3000)

In [ ]:
client.info()

In [ ]:
from enfobench.evaluation import cross_validate

crossval_df = cross_validate(
    client,
    dataset,
    start_date=pd.Timestamp("2013-06-01T10:00:00"),
    end_date=pd.Timestamp("2013-07-01T00:00:00"),
    horizon=pd.Timedelta("38 hours"),
    step=pd.Timedelta("1 day"),
)

In [ ]:
crossval_df.head()

In [ ]:
import matplotlib.animation as animation

plt.rcParams["animation.html"] = "jshtml"

In [ ]:
plt.ioff()
fig, ax = plt.subplots(figsize=(12, 5))

cutoff_dates = crossval_df.cutoff_date.unique()


def animate_forecast(t):
    plt.cla()

    cutoff_date = cutoff_dates[t]
    history = dataset.get_history(cutoff_date)
    forecast = (
        crossval_df.loc[crossval_df.cutoff_date == cutoff_date].set_index("timestamp").drop(columns=["cutoff_date"])
    )

    ax.plot(history.index, history.y)
    ax.plot(forecast.index, forecast.yhat)
    ax.set_xlim(cutoff_dates[0] - pd.Timedelta('7D'), crossval_df.timestamp.max())
    ax.set_ylabel("Energy (kWh)")
    ax.set_title(f"Predicted energy consumption at {cutoff_date}", fontsize="large", loc="left")


ani = animation.FuncAnimation(fig, animate_forecast, frames=len(cutoff_dates))
ani

In [ ]:
plt.ion()
plt.close()

# Metrics

In [ ]:
from enfobench.evaluation import evaluate_metrics
from enfobench.evaluation.metrics import mean_absolute_error, mean_bias_error

In [ ]:
overall_metrics = evaluate_metrics(crossval_df, metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error})
overall_metrics

In [ ]:
metrics_per_cutoff = evaluate_metrics(
    crossval_df,
    metrics={"MAE": mean_absolute_error, "MBE": mean_bias_error},
    groupby="cutoff_date",
)
metrics_per_cutoff.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
metrics_per_cutoff.set_index("cutoff_date")[["MAE", "MBE"]].plot(ax=ax)
ax.set_xlabel("Cutoff date")
ax.set_ylabel("Energy (kWh)")
ax.set_title("Prediction metrics per cutoff point", fontsize="large", loc="left")
plt.show()